# Notebook para o PAN - Atribuição Autoral - 2018

In [1]:
%matplotlib inline
#python basic libs
import os;
from os.path import join as pathjoin;

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

import re;
import glob;
import json;
import codecs;
from collections import defaultdict;
import pprint;


from pprint import pprint
from time import time
import logging


#data analysis libs
import numpy as np;
import pandas as pd;
from pandas.plotting import scatter_matrix;
import matplotlib.pyplot as plt;
import random;

#machine learning libs
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

#preprocessing and transformation
from sklearn.preprocessing import normalize, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler;
from sklearn.preprocessing import LabelBinarizer;
from sklearn.decomposition import PCA;
from sklearn.metrics.pairwise import cosine_similarity;


from sklearn.base import BaseEstimator, ClassifierMixin

#classifiers
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.feature_selection import RFE,SelectFpr,SelectPercentile, chi2;

#
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#model valuation
from sklearn.model_selection import train_test_split;
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score;

In [2]:
import seaborn as sns;
sns.set(color_codes=True);

In [3]:
import platform;
import sklearn;
import scipy;
import spacy;

print("|%-15s|%-40s|"%("PACK","VERSION"))
print("|%-15s|%-40s|"%('-'*15,'-'*40))
print('\n'.join(
    "|%-15s|%-40s|" % (pack, version)
    for pack, version in
    zip(['SO','NumPy','SciPy','Scikit-Learn','seaborn','spacy'],
        [platform.platform(), np.__version__, scipy.__version__, sklearn.__version__, sns.__version__, spacy.__version__])

))

|PACK           |VERSION                                 |
|---------------|----------------------------------------|
|SO             |Darwin-18.2.0-x86_64-i386-64bit         |
|NumPy          |1.15.4                                  |
|SciPy          |1.1.0                                   |
|Scikit-Learn   |0.20.1                                  |
|seaborn        |0.9.0                                   |
|spacy          |2.0.16                                  |


In [4]:
np.set_printoptions(precision=4)
pd.options.display.float_format = '{:,.4f}'.format

### paths configuration

### on google

from google.colab import drive
drive.mount('/gdrive')

!ls '/gdrive/My Drive/data_repository'

baseDir = '/gdrive/My Drive/data_repository';
modelName = 'char1';

inputDir= pathjoin(baseDir,'pan18aa');
outputDir= pathjoin(baseDir,'out',modelName);

os.makedirs(outputDir, exist_ok=True)


def transferCode(filename):
    with open(os.path.join('/gdrive/My Drive','Colab Notebooks',filename)) as fin:
        with open(filename,'w') as fout:
            fout.write(fin.read());

transferCode('pan.py')
transferCode('sklearnExtensions.py')

!ls 

In [5]:
### on local

In [6]:
baseDir = '/Users/joseeleandrocustodio/dataScienceNotebook/PANAA2018';

inputDir= pathjoin(baseDir,'pan18aa');
outputDir= pathjoin(baseDir,'out',"oficial");
if not os.path.exists(outputDir):
    os.mkdir(outputDir);

## Data Ingestion

In [7]:
import pan
import sklearnExtensions
from sklearnExtensions import DenseTransformer
from sklearnExtensions import POSTagTransformer

In [8]:
problems = pan.readCollectionsOfProblems(inputDir);

In [9]:
pd.DataFrame(problems)

,candidates,candidates_folder_count,encoding,language,problem,unknown
0,"[[graceful ones.\n\n""One more,"" Marvelous said...",20,UTF-8,en,problem00001,"[[after all, his best friends. And what in the..."
1,"[[a mission.""\n\nJensen just raises an eyebrow...",5,UTF-8,en,problem00002,"[[“Potter was attractive,” Draco thought, sigh..."
2,[[qui l'avait tué mais tout était de la faute ...,20,UTF-8,fr,problem00003,[[son réveil. Sa main pulse et Draco frotte l'...
3,[[. Le canapé est vide et lorsqu'il passe deva...,5,UTF-8,fr,problem00004,"[[abasourdie.\n\nTout d'abord, elle crut que s..."
4,"[[Eppure lui la mappa l’aveva stampata, dannaz...",20,UTF-8,it,problem00005,[[– Oh. Cazzo.\nSirius era così sconvolto che ...
5,[[Yato ha trovato una lettera sul suo comodino...,5,UTF-8,it,problem00006,"[[così la tua vista, Moony?\n– Cercavo di esse..."
6,[[zmienił zdanie. Niech się stworzonko pobawi....,20,UTF-8,pl,problem00007,"[[dawniej pełna radości i ciepła, a teraz wiec..."
7,"[[Słowem, które Sherlock najczęściej słyszał w...",5,UTF-8,pl,problem00008,"[[, uderzającego o żebra niczym dzwon- niemal ..."
8,[[pero no lo ama como ama a Guignol –explicó e...,20,UTF-8,sp,problem00009,[[–La nariz puntiaguda del elfo casi rozaba el...
9,"[[incapaz de señalar un momento exacto, un pun...",5,UTF-8,sp,problem00010,[[tan parecidas hizo que su trasero latiese de...


In [10]:
print('\u0104')

Ą


In [11]:
distortions = [
    (r"[A-Z]"          ,'U','Upcase '),
    (r"[a-z]"          ,'l','lowercase'),
    (r"[À-ÿĄ-ż]"       ,'d','Diacrit'),
    (r"\d"             ,'D','Digits'),
    (r"[\n\r\t]"       ,'E','Paragraph control'),
    (r"\–|\-|\_|\—"    ,'H','Hyphen'),
    (r"\"|\'|’|[“”]"   ,'Q','quotation'),
    (r"\s"             ,'S','Spaces'),
    (r"\(|\)|\[|\]\{|\}|\«|\»",'G','Groups'),
    (r"…",'','...'),
    (r"\.|\,|[?]|\!|\:|\;|\„|\*|\/"   ,'P','Punctuation'),
    (r"[^aUldDEHQSGP]", 'X','Unkown'),
]

distortions = [(re.compile(d[0]), d[1],d[2]) for d in distortions]

In [12]:
t = problems[8]['candidates'][0][0]
for d in distortions:
    t = d[0].sub(d[1],t)
print(t)
counts = defaultdict(int);
for c in t:
    counts[c]+=1;
pprint (counts)

llllSllSllSlllSllllSlllSlSUllllllSHlllllldSllSlldlSlllSlllllllllSlllSlllllllllllllSlllllllllSllSllllSllSllllllPEEHXUSldllSlllllSlllSllSllSlllPSHUlllllllllSllSllllllldSlSllSlllSllSlllldlSllSlldlllSlllSllllllSllllldlPEEHUllllllSllSllSlllldPSllllSlllSlllSllSlllllllSHUlllldSllllldSllllSllSlllllllSllSllllllSllSlllllllllHPSXUlllPSUllllllSllSlllSllSllllllllPSlSllSllSllllllllllSlSlllllSllSllSlllllllllSllllllSllllllSlSllllllSlSldlSllllllPEEXUllllllSllldlSllllllSllSlllllSllllSlllllSlllllSlllSdlPSUldSlllllllllPSlllldSUlllllllllPSllSllSlllldlSllllllllSlllSlllSlllSlllllllllSllllSllllllllllSllllSllSlllllllHlllllHllllllllSlllSllllPSUlllSldSllllldlSlllSllllllSllSdlSlllllSllSlllSlllllllSlllSllllllSllllllSllSlllldllllSlSUlllldSllllllllllSllllllllllllPSlllSllSlllSllSlllllllSlllllSlllllllSllllllllSllSlllllllllSlllSllllllSlllPSlllSlllSllllSllldlSlllldlllSllSlllllllllSllllllSlllSllllllPSUllSllllSllllPSllSUlllldSlSUllllllSllldllSlllllSlllSlllllPSlllldlSlllllSlllSlllSdlPSUlllllllllPSlllllSldlSlllSllSllllllllS

In [13]:
class DistortionTransformer(BaseEstimator):
    def __init__(self,verbose=False):
        self.verbose = verbose;
        
    def _distortion(self,doc):
        for d in distortions:
            doc = d[0].sub(d[1],doc);
        return doc;

    def transform(self, X, y=None):
        t0 = time()
        if self.verbose:
            print('filtered in', end=' ')

        X = [self._distortion(doc) for doc in X]
        
        if self.verbose:
            print(" %0.3fs" % (time() - t0))
        return np.array(X);       

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [14]:
languages={
    'en':'inglesa',
    'sp':'espanhola',
    'it':'italiana',
    'pl':'polonesa',
    'fr':'francesa'
}

### analisando os demais parametros

In [17]:
def runML(problem):
    import gc;
    gc.collect();
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']));
    
    train_docs, train_labels, _   = zip(*problem['candidates'])
    problem['training_docs_size'] = len(train_docs);
    test_docs, _, test_filename   = zip(*problem['unknown'])
    
    t0 = time()
    
    pipeline = Pipeline([
        ('filter',DistortionTransformer()),
        ('vect', TfidfVectorizer(
                analyzer='char',
                min_df=0.01,
                lowercase=False,
                norm='l2',
                sublinear_tf=True
                )
        ),
        ('scaler', MaxAbsScaler()),
        ('dense',  DenseTransformer()),
        ('transf', PCA(0.999)),
        ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
    ])
    
    
    # uncommenting more parameters will give better exploring power but will
    # increase processing time in a combinatorial way
    parameters = {
        'vect__ngram_range':((1,3),(1,5),(2,5),(2,4),(3,5)),
        'vect__norm':('l1','l2',None),
        'clf__C':(0.01,0.1,1,10),
    }
    
    grid_search = GridSearchCV(pipeline,
                               parameters,
                               cv=3,
                               iid=False,
                               n_jobs=-1,
                               verbose=True,
                               scoring='f1_macro')
    
    t0 = time()
    grid_search.fit(train_docs, train_labels)
    
    print("Gridsearh %0.3fs" % (time() - t0), end=' ')

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t'%s': %r," % (param_name, best_parameters[param_name]))
        
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
    
    # Writing output file
    out_data=[]
    for i,v in enumerate(test_pred):
        out_data.append({'unknown-text': test_filename[i],'predicted-author': v})
    answerFile = pathjoin(outputDir,'answers-'+problem['problem']+'.json');
    with open(answerFile, 'w') as f:
        json.dump(out_data, f, indent=4)
    
    
    #calculating the performance using PAN evaluation code
    f1,precision,recall,accuracy=pan.evaluate(
                pathjoin(inputDir, problem['problem'], 'ground-truth.json'),
                answerFile)
    
    return {
                'problem-name'  :       problem['problem'],
                "language"      :       problem['language'],
                'macro-f1'       : round(f1,3),
                'macro-precision': round(precision,3),
                'macro-recall'   : round(recall,3),
                'micro-accuracy' : round(accuracy,3),
                
        }, grid_search.cv_results_,best_parameters, grid_search.best_estimator_;

In [19]:
result = [];
cv_result = [];
best_parameters = [];
estimators = [];
with warnings.catch_warnings():
    warnings.filterwarnings("ignore");
    for problem in problems:
        i = int(re.sub('\D+','',problem['problem']));
        r, c, b, e = runML(problem);
        result.append(r);
        cv_result.append(c);
        estimators.append(e);
        b['problem'] = problem['problem'];
        best_parameters.append(b);
os.system( "say finished" )


Problem: problem00001,  language: en, 
Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.0min finished


Gridsearh 179.980s Best score: 0.643
Best parameters set:
	'clf__C': 0.1,
	'vect__ngram_range': (1, 5),
	'vect__norm': 'l1',

Problem: problem00002,  language: en, 
Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   47.1s finished


Gridsearh 47.321s Best score: 0.956
Best parameters set:
	'clf__C': 1,
	'vect__ngram_range': (3, 5),
	'vect__norm': 'l2',

Problem: problem00003,  language: fr, 
Fitting 3 folds for each of 60 candidates, totalling 180 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.8min finished


Gridsearh 227.123s Best score: 0.688
Best parameters set:
	'clf__C': 1,
	'vect__ngram_range': (1, 5),
	'vect__norm': 'l2',

Problem: problem00010,  language: sp, 
Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.0min finished


Gridsearh 60.699s Best score: 0.750
Best parameters set:
	'clf__C': 1,
	'vect__ngram_range': (3, 5),
	'vect__norm': 'l1',


0

In [20]:
df=pd.DataFrame(result)[['problem-name',
                     "language",
                     'macro-f1','macro-precision','macro-recall' ,'micro-accuracy']]

In [21]:
df

,problem-name,language,macro-f1,macro-precision,macro-recall,micro-accuracy
0,problem00001,en,0.3870,0.3740,0.5350,0.5520
1,problem00002,en,0.4100,0.4110,0.5330,0.4290
2,problem00003,fr,0.5900,0.6170,0.6520,0.5920
3,problem00004,fr,0.6870,0.6920,0.7730,0.7140
4,problem00005,it,0.4570,0.4490,0.5790,0.5000
5,problem00006,it,0.4400,0.5280,0.5680,0.6300
6,problem00007,pl,0.3180,0.3480,0.3590,0.3790
7,problem00008,pl,0.7870,0.7330,0.9330,0.8000
8,problem00009,sp,0.5080,0.5240,0.5390,0.5380
9,problem00010,sp,0.6310,0.6490,0.6910,0.7030


In [22]:
print(df[["macro-f1"]].reset_index().to_latex(index=False).replace("     "," "))

\begin{tabular}{rr}
\toprule
 index &  macro-f1 \\
\midrule
 0 &    0.3870 \\
 1 &    0.4100 \\
 2 &    0.5900 \\
 3 &    0.6870 \\
 4 &    0.4570 \\
 5 &    0.4400 \\
 6 &    0.3180 \\
 7 &    0.7870 \\
 8 &    0.5080 \\
 9 &    0.6310 \\
\bottomrule
\end{tabular}



In [23]:
languages={
    'en':'inglesa',
    'sp':'espanhola',
    'it':'italiana',
    'pl':'polonesa',
    'fr':'francesa'
}

In [24]:
cv_result2 = [];
dfCV = pd.DataFrame();
for i, c in enumerate(cv_result):
    temp = pd.DataFrame(c);
    temp['problem'] = result[i]['problem-name'];
    temp['language'] = languages[result[i]['language']]
    dfCV = dfCV.append(temp);

for p in [
    'mean_test_score','std_test_score','mean_train_score',   
    'split0_test_score','split0_train_score',
    'split1_test_score','split1_train_score',
    'split2_test_score','split2_train_score']:
    dfCV[p]=dfCV[p].astype(np.float32);

    
dfCV =dfCV[[
    'problem',
    'language',
    'rank_test_score',
    'param_clf__C',
    'param_vect__norm',
    'param_vect__ngram_range',
    'mean_test_score',   
    'std_test_score',
    'mean_train_score',   

    'split0_test_score','split0_train_score',
    'split1_test_score','split1_train_score',
    'split2_test_score','split2_train_score',

    'mean_score_time',
    'mean_fit_time',
    'std_fit_time',
    'std_score_time',
    'std_train_score',
]];

dfCV.rename(columns={
    'param_clf__C':'regularization',
    'param_vect__norm':'norm',
    'param_vect__ngram_range':'ngram_range',
},inplace=True);

def nrange(x):
    return sum([i for i in range(x+1)])

dfCV['rangeLen'] = dfCV.ngram_range.apply(lambda x: nrange(x[1])-nrange(x[0]-1))

#print('\',\n\''.join(dfCV.columns))

In [25]:
dfCV.to_csv('PANAA2018_DISTORTION.csv', index=False)

In [26]:
dfCV = pd.read_csv('PANAA2018_DISTORTION.csv', na_values='')

In [27]:
(dfCV[dfCV.rank_test_score == 1])[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'std_test_score',
    'ngram_range',
    'rangeLen',
    'regularization',
    'norm',
]
].sort_values(by=[
    'problem',
    'mean_test_score',
    'regularization',
    'norm',
    'rangeLen',
    'ngram_range',
], ascending=[True, False,True,True, True,True]).head(10)

,problem,language,rank_test_score,mean_test_score,std_test_score,ngram_range,rangeLen,regularization,norm
18,problem00001,inglesa,1,0.6425,0.0724,"(1, 5)",15,0.1000,l1
103,problem00002,inglesa,1,0.9556,0.0629,"(3, 5)",12,1.0000,l2
118,problem00002,inglesa,1,0.9556,0.0629,"(3, 5)",12,10.0000,l2
164,problem00003,francesa,1,0.6400,0.0253,"(3, 5)",12,1.0000,NaN
192,problem00004,francesa,1,0.7148,0.0209,"(3, 5)",12,0.0100,l1
186,problem00004,francesa,1,0.7148,0.0209,"(2, 5)",14,0.0100,l1
294,problem00005,italiana,1,0.5800,0.0235,"(2, 4)",9,10.0000,l1
327,problem00006,italiana,1,0.9060,0.0180,"(3, 5)",12,0.1000,l1
328,problem00006,italiana,1,0.9060,0.0180,"(3, 5)",12,0.1000,l2
322,problem00006,italiana,1,0.9060,0.0180,"(2, 5)",14,0.1000,l2


In [28]:
pd.set_option('display.max_colwidth', -1)
t=(dfCV)[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'std_test_score',
    'ngram_range',
    'rangeLen',
    'regularization',
    'norm',
]
].sort_values(by=[
    'problem',
    'mean_test_score',
    'regularization',
    'norm',
    'rangeLen',
    'ngram_range',
], ascending=[True, False,True,True, True,True])
t

,problem,language,rank_test_score,mean_test_score,std_test_score,ngram_range,rangeLen,regularization,norm
18,problem00001,inglesa,1,0.6425,0.0724,"(1, 5)",15,0.1000,l1
27,problem00001,inglesa,2,0.6421,0.0857,"(3, 5)",12,0.1000,l1
33,problem00001,inglesa,3,0.6403,0.0562,"(1, 5)",15,1.0000,l1
51,problem00001,inglesa,3,0.6403,0.0562,"(2, 5)",14,10.0000,l1
28,problem00001,inglesa,5,0.6386,0.0815,"(3, 5)",12,0.1000,l2
22,problem00001,inglesa,5,0.6386,0.0815,"(2, 5)",14,0.1000,l2
19,problem00001,inglesa,5,0.6386,0.0815,"(1, 5)",15,0.1000,l2
21,problem00001,inglesa,8,0.6379,0.0819,"(2, 5)",14,0.1000,l1
36,problem00001,inglesa,9,0.6355,0.0627,"(2, 5)",14,1.0000,l1
23,problem00001,inglesa,10,0.6352,0.0666,"(2, 5)",14,0.1000,NaN


In [29]:
pd.options.display.precision = 3
pd.options.display.max_colwidth = -1
t =(dfCV)[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'std_test_score',
    'ngram_range',
    'rangeLen',
    'regularization',
    'norm',
]
].sort_values(by=[
    'problem',
    'mean_test_score',
    'regularization',
    'norm',
    'rangeLen',
    'ngram_range',
], ascending=[True, False,True,True, True,True])\
.groupby(['problem']).first().drop(columns='rank_test_score').reset_index()


print(u"\\begin{table}[h]\n\\centering\n\\caption{Medida F1 para os parâmetros }")

print(re.sub(r'[ ]{2,}',' ',t.to_latex()))
print ("\label{tab:modelocaracter}")
print(r"\end{table}")

\begin{table}[h]
\centering
\caption{Medida F1 para os parâmetros }
\begin{tabular}{lllrrlrrl}
\toprule
{} & problem & language & mean\_test\_score & std\_test\_score & ngram\_range & rangeLen & regularization & norm \\
\midrule
0 & problem00001 & inglesa & 0.6425 & 0.0724 & (1, 5) & 15 & 0.1000 & l1 \\
1 & problem00002 & inglesa & 0.9556 & 0.0629 & (3, 5) & 12 & 1.0000 & l2 \\
2 & problem00003 & francesa & 0.6400 & 0.0253 & (3, 5) & 12 & 1.0000 & l2 \\
3 & problem00004 & francesa & 0.7148 & 0.0209 & (3, 5) & 12 & 0.0100 & l1 \\
4 & problem00005 & italiana & 0.5800 & 0.0235 & (2, 4) & 9 & 10.0000 & l1 \\
5 & problem00006 & italiana & 0.9060 & 0.0180 & (3, 5) & 12 & 0.1000 & l1 \\
6 & problem00007 & polonesa & 0.5891 & 0.0790 & (1, 5) & 15 & 10.0000 & l2 \\
7 & problem00008 & polonesa & 0.7800 & 0.0984 & (2, 4) & 9 & 0.1000 & l2 \\
8 & problem00009 & espanhola & 0.6883 & 0.0347 & (1, 5) & 15 & 1.0000 & l2 \\
9 & problem00010 & espanhola & 0.7505 & 0.1097 & (3, 5) & 12 & 1.0000 & l1 \\
\

In [30]:
# TESTS

In [31]:
def d(estimator, n_features=15):
    from IPython.display import Markdown, display, HTML
    names = np.array(estimator.named_steps['vect'].get_feature_names());
    classes_ = estimator.named_steps['clf'].classes_;
    a = estimator.named_steps['clf'].coef_
    b = estimator.named_steps['transf'].components_
    c = estimator.named_steps['vect'].idf_
    weights = a.dot(b) * c
    
    def tag(tag, content, attrib=''):
        if attrib != '':
            attrib = ' style="' + attrib+'"'; 
        return ''.join(['<',tag,attrib,' >',content,'</',tag,'>']);
    
    def color(baseColor, intensity):
        r,g,b = baseColor[0:2],baseColor[2:4],baseColor[4:6]
        r,g,b = int(r, 16), int(g, 16), int(b, 16)
        
        f= (1-np.abs(intensity))/3;
        r = r + int((255-r)*f)
        g = g + int((255-g)*f)
        b = b + int((255-b)*f)
        rgb = '#%02x%x%x' % (r, g, b);
        #print(baseColor,rgb,r,g,b,intensity,f)
        return rgb
    
    
    spanStyle ='border-radius: 5px;margin:2px 4px;padding:2px; color:#FFF;display:inline-block;';
    
    lines = '<table>'+tag('thead',tag('th','Classes')+tag('th','positive')+tag('th','negative'))
    lines += '<tbody>'
    for i,c in enumerate(weights):
        c = np.round(c / np.abs(c).max(),2);
        positive = names[np.argsort(-c)][:n_features];
        positiveV = c[np.argsort(-c)][:n_features]
        negative = names[np.argsort(c)][:n_features];
        negativeV = c[np.argsort(c)][:n_features]
        
        lines += tag('tr',
                     tag('td', re.sub('\D0*','',classes_[i]))
                     + tag('td',''.join([tag('span',d+' '+str(v),spanStyle+'background:'+color('51A3DD',v)) for d,v in zip(positive,positiveV)]))
                     + tag('td',''.join([tag('span',d+' '+str(v),spanStyle+'background:'+color('DD5555',v)) for d,v in zip(negative,negativeV)]))
                    )
    lines+= '</tbody></table>'
        
    display(HTML(lines))
    #print(lines)
        
d(estimators[0])

1,lPG 1.0PG 0.96lPGE 0.95PEEG 0.9PGE 0.9llPG 0.9lPEEG 0.9EEG 0.87EG 0.86GE 0.85lllPG 0.85PEEGl 0.85lPGEE 0.83EEGl 0.79EGl 0.79,EUlll -0.16PQ -0.15QU -0.15EQUll -0.15SllHl -0.14SlSUl -0.14lllPQ -0.14QUl -0.14QUlSU -0.14QUll -0.14QUllP -0.14lPQ -0.14llPQ -0.14lEE -0.14ES -0.14
2,PPPSU 1.0QllQP 0.98PPSUl 0.96lSHH 0.91SHH 0.91HHQ 0.89llSHH 0.89SHHS 0.89lSHHS 0.89PPSU 0.88HHS 0.87lHHQ 0.86SHHSl 0.83HHSl 0.81HHSll 0.81,QlllS -0.25EQUQl -0.22EEQUQ -0.22EQUQ -0.22QlPSl -0.21PPll -0.2PPl -0.2UlPQ -0.2PSllP -0.2lQS -0.19PPPl -0.19llQS -0.19SQllS -0.19PPPll -0.19UPQ -0.18
3,PEEEU 1.0QEEEU 0.81SEEEU 0.74QEEEQ 0.74PlllS 0.57SUlSU 0.54PUllS 0.54PSSSS 0.53SPSUl 0.52lSPSU 0.52llSSU 0.5lSSU 0.5SPSU 0.49UlSU 0.41Plll 0.4,EEEE -0.26EEEEU -0.25SEEEE -0.22PEEEE -0.22QlPSl -0.21UlPSl -0.2PEEQU -0.19UlP -0.19PEEQ -0.19QUlQ -0.19Hllll -0.19lPEEQ -0.19SUlP -0.19lQllS -0.18SlllH -0.18
4,SUlPQ 1.0lSQUS 0.79EEQH 0.79EQH 0.79llSQU 0.68lSQU 0.68QHll 0.67SlHQE 0.67QHl 0.67QHlll 0.67HQEEU 0.66QlQPS 0.65QlQP 0.65QH 0.64QlSUP 0.64,PP -0.36lSHSU -0.34UlSU -0.32EUlSU -0.3PPP -0.3UlSUl -0.29lPP -0.29PPS -0.29llPP -0.28lPPP -0.28EQUSl -0.28EQUS -0.28PPPS -0.28llPPP -0.27SUQll -0.27
5,PEEPP 1.0EPPPE 1.0lSlHl 0.89SlHll 0.89QEEUS 0.89SlHl 0.89PEEP 0.84lSlH 0.79SlH 0.79llSlH 0.79lSUUP 0.77SUUP 0.74EEPPP 0.73EEPP 0.73EPP 0.73,lSQll -0.31lSQl -0.3SllPE -0.29llSQl -0.27lSQ -0.26SQUlS -0.26llSQ -0.25QlPSl -0.23SX -0.23EUlSU -0.22lSXll -0.21PPS -0.21QUlSU -0.21lSUP -0.21lllSQ -0.21
6,SSSSU 1.0SSSQU 0.96PEESS 0.96SSSSQ 0.95QEESS 0.95SSSQ 0.95EESSS 0.94SSSUl 0.94EESS 0.94ESSSS 0.93ESS 0.93ESSS 0.93SSSU 0.9SSSSS 0.89SEUl 0.89,PSQl -0.35PSQll -0.35lPSQl -0.35SQllS -0.32SEEQ -0.29SEEQU -0.29UlQll -0.29SUllP -0.28SEE -0.28HQ -0.28SllH -0.27UlQ -0.26lSllH -0.26UlQl -0.25SUlQ -0.25
7,SHSQE 1.0HSQE 1.0HSQEE 1.0SQE 0.93SQEE 0.93QUSUU 0.91SHSQ 0.9lSHSQ 0.9SQEEU 0.87SQllS 0.87QUSU 0.84DE 0.82lD 0.8HSQ 0.8HEEUS 0.79,lPSUQ -0.37PSUQ -0.37PSUQl -0.36PP -0.28lPP -0.27llPP -0.26UQlll -0.25PPS -0.24PPP -0.23HQSUl -0.22SUlQl -0.22lPPP -0.22QEEUS -0.22lllPP -0.22QlPSl -0.22
8,PEESU 1.0XX 0.98EXXXX 0.95XXXXE 0.95XXXX 0.95XXX 0.93ESU 0.89EESU 0.89DDDP 0.88PEESQ 0.87ESQ 0.84EESQ 0.84EXXX 0.81XXEE 0.81EXX 0.81,QUlSU -0.27EQUlQ -0.24UlSU -0.24llQll -0.23lQllS -0.22lQll -0.21UlSUS -0.2QllS -0.2lSQll -0.2QllSl -0.2SllH -0.2PEEQU -0.2SUllP -0.2lSQl -0.2lSQ -0.2
9,lQEEU 1.0lQEE 0.72lQE 0.72PPQSl 0.67QlllP 0.64lllQE 0.58llQEE 0.55llQE 0.55UlE 0.55UlEE 0.55PSQUP 0.54SQUP 0.54lPQQS 0.53PEEHH 0.53EEHHH 0.53,PSUQl -0.29EEQUQ -0.28EQUQl -0.28EQUQ -0.28HQSU -0.27HQSUl -0.27lllQP -0.25HQ -0.23HQS -0.22UllPS -0.22QUlSU -0.22lSUP -0.21SUllQ -0.2QlSU -0.2SUQlS -0.2
10,QPSll 1.0QPSl 1.0SllQP 0.96UllQP 0.93lQPSl 0.9lPQPS 0.89lPQP 0.89llPQP 0.89PQP 0.86PQPS 0.86QPS 0.83lQPSU 0.81lPPPE 0.79QPSU 0.78QPSUl 0.78,PQSll -0.22PQSl -0.22lPQSl -0.21QSllS -0.21SllH -0.19X -0.18QEEU -0.18PQEEU -0.18lSllH -0.18QEEUl -0.17QEEQU -0.17HSlll -0.17QEEQ -0.17UlQll -0.17PQEEQ -0.17
11,HSllQ 1.0QlHQ 0.72lQlHQ 0.72PQSlS 0.6QSlSl 0.53QSlS 0.53QlHQE 0.51PPSll 0.47PPPSl 0.45QUlPP 0.45PPSl 0.45lHQSU 0.44UlPPP 0.42DSH 0.41DSHS 0.41,QUllQ -0.2QlllS -0.17UPQ -0.14HSUSl -0.14HSUS -0.14QSE -0.13PQSEE -0.13PQSE -0.13PSEEQ -0.13lPQSE -0.13PPll -0.13PPl -0.13SX -0.13QSEE -0.13QSUlS -0.12
